In [163]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import click
from pathlib import Path
import preprocessor as p
import re

# text Manipulation for TWIVER CMU

The next few lines are for our manipulation and not part of the examples for BERT below

In [181]:
file_day = 'ukraine_day_processed.csv'
turk_file_day = 'ukraine_day_processed_turked.csv'

file_week = 'ukraine_week_processed.csv'
turk_file_week = 'ukraine_week_processed_turked.csv'

file_month = 'ukraine_month_processed.csv'
turk_file_month= 'ukraine_month_processed_turked.csv'

path = Path.cwd()
text_Dir = path.joinpath('ukraine_processed')
turk_Dir = path.joinpath('ukraine_turked')
by_hand_Dir = turk_Dir.joinpath('ukraine_turked_byhand')

file_Dir_day = text_Dir.joinpath(file_day)
turkFile_Dir_day = by_hand_Dir.joinpath(turk_file_day)

file_Dir_week = text_Dir.joinpath(file_week)
turkFile_Dir_week = by_hand_Dir.joinpath(turk_file_week)

file_Dir_month = text_Dir.joinpath(file_month)
turkFile_Dir_month = by_hand_Dir.joinpath(turk_file_month)
#print(text_Dir)
tweets_processed_day = pd.read_csv(file_Dir_day, dtype='str' , names=['id', 'tweet'])
tweets_turked_day = pd.read_csv(turkFile_Dir_day, dtype='str' , names=['non-id', 'tweet', 'rating'])

tweets_processed_week = pd.read_csv(file_Dir_week, dtype='str' , names=['id', 'tweet'])
tweets_turked_week = pd.read_csv(turkFile_Dir_week, dtype='str' , names=['non-id', 'tweet', 'rating'])

tweets_processed_month = pd.read_csv(file_Dir_month, dtype='str' , names=['id', 'tweet'])
tweets_turked_month = pd.read_csv(turkFile_Dir_month, dtype='str' , names=['non-id', 'tweet', 'rating'])


In [182]:
day_merge = pd.merge(tweets_processed_day, tweets_turked_day, 'right', 'tweet')
day_merge = day_merge.drop_duplicates(subset=['tweet'])
day_merge = day_merge.loc[:, ['id', 'tweet', 'rating']]
values = ['DY', 'PY' , 'UC' , 'PN', 'DN'] 
day_merge = day_merge[day_merge.rating.isin(values) ==True ]
day_outpath = turk_Dir.joinpath('ukraine_day_turked')
print(day_merge.head(10))
print(len(day_merge.drop_duplicates()))
day_merge.to_csv(day_outpath, header=False, index=False)

                     id                                              tweet  \
0   1496273443754496003  @SocComdeeznuts @theschwasound @LobbyDob No it...   
1   1496273551048986627  @theRedacted0ne &gt;implying itll happen\nruss...   
2   1496273438381686790  Nothing but lies.  Ukraine claims there are no...   
3   1496273417372373000  Just because anonymous US officials have been ...   
9   1496273394383441926  @JackPosobiec We are being played.  #Ukraine #...   
10  1496273370907828224  There is worry that #Russia will invade #Ukrai...   
11  1496273362737373186  @freedomrideblog I didn’t realize the us put 1...   
12  1496273357117206528  Putin Invades Ukraine (But Not Really) https:/...   
13  1496273331703599106  @NBCNews Stop the lies about Putin and Russia....   
14  1496273597719056386  @Denis_Buchholz @Cinderoak This is when Putin'...   

   rating  
0      UC  
1      DN  
2      DN  
3      DN  
9      DN  
10     DN  
11     DN  
12     DN  
13     DN  
14     DY  
1511


In [183]:
day_merge['TimeFrame'] = 'day'
day_merge.head(10)

,id,tweet,rating,TimeFrame
0,1496273443754496003,@SocComdeeznuts @theschwasound @LobbyDob No it...,UC,day
1,1496273551048986627,@theRedacted0ne &gt;implying itll happen\nruss...,DN,day
2,1496273438381686790,Nothing but lies. Ukraine claims there are no...,DN,day
3,1496273417372373000,Just because anonymous US officials have been ...,DN,day
9,1496273394383441926,@JackPosobiec We are being played. #Ukraine #...,DN,day
10,1496273370907828224,There is worry that #Russia will invade #Ukrai...,DN,day
11,1496273362737373186,@freedomrideblog I didn’t realize the us put 1...,DN,day
12,1496273357117206528,Putin Invades Ukraine (But Not Really) https:/...,DN,day
13,1496273331703599106,@NBCNews Stop the lies about Putin and Russia....,DN,day
14,1496273597719056386,@Denis_Buchholz @Cinderoak This is when Putin'...,DY,day


In [184]:
week_merge = pd.merge(tweets_processed_week, tweets_turked_week, 'right', 'tweet')
week_merge = week_merge.drop_duplicates(subset=['tweet'])
week_merge = week_merge.loc[:, ['id', 'tweet', 'rating']]
values = ['DY', 'PY' , 'UC' , 'PN', 'DN'] 
week_merge = week_merge[week_merge.rating.isin(values) ==True ]
week_outpath = turk_Dir.joinpath('ukraine_week_turked')
print(week_merge.head(10))
print(len(week_merge))
week_merge.to_csv(week_outpath, header=False, index=False)

                      id                                              tweet  \
0    1494099270399627266  @POTUS you and nato are the ones violating Rus...   
1    1494099269858603008  BREAKING: The Biden adminsitration says within...   
27   1494099268176527360  #UPDATE: #Russia has been building bridges and...   
28   1494099264636481540  If Russia launches an invasion of Ukraine, mil...   
29   1494099264452059142  The Biden administration telling US citizens t...   
30   1494099263567061003  A senior Central African Republic military off...   
40   1494099262526873602  NATO chief, US, and open source show no pullba...   
48   1494099259116904448  Russia has added about 7,000 troops along Ukra...   
147  1494099258773065732  As much as I loathe Putin, he has shown up Bid...   
148  1494099257388896258  Maria Zakharova, the spokeswoman for Russia's ...   

    rating  
0       UC  
1       PY  
27      PY  
28      PY  
29      PY  
30      PY  
40      PY  
48      PY  
147     UC  


In [200]:
month_merge = pd.merge(tweets_processed_month, tweets_turked_month, 'right', 'tweet')
month_merge = month_merge.drop_duplicates(subset=['tweet'])
month_merge = month_merge.loc[:, ['id', 'tweet', 'rating']]
values = ['DY', 'PY' , 'UC' , 'PN', 'DN'] 
month_merge = month_merge[month_merge.rating.isin(values) ==True ]
month_outpath = turk_Dir.joinpath('ukraine_month_turked')
print(month_merge.head(10))
print(len(month_merge))
month_merge.to_csv(month_outpath, header=False, index=False)

                      id                                              tweet  \
1    1485039574556438530  @disclosetv Poppycock. Nothing to see here. Me...   
2    1485039574288150530  In a bid to continue the attacks, @AbiyAhmedAl...   
3    1485039574027886593  BREAKING NEWS\n\nPUTIN PREPARING TO INSTALL PR...   
21   1485039571796692999  Hope these planes in Ukraine have better wheel...   
83   1485039570576023555  LONDON (AP) — British government accuses Russi...   
87   1485039570169303043  United States is creating this crisis by encou...   
88   1485039569825374209  Russia aims to replace Ukraine government with...   
93   1485039567912816640  What is Joe Biden’s plan to stop Russia from i...   
122  1485039566285422606  I think we should start paying far more attent...   
127  1485039565366706178  THERE SHOULD BE NO AMERICAN INVOLVEMENT IN UKR...   

    rating  
1       DY  
2       DY  
3       DY  
21      DY  
83      DY  
87      DY  
88      DY  
93      DY  
122     DY  


In [201]:
week_merge['TimeFrame'] = 'week'
week_merge.head(10)

,id,tweet,rating,TimeFrame
0,1494099270399627266,@POTUS you and nato are the ones violating Rus...,UC,week
1,1494099269858603008,BREAKING: The Biden adminsitration says within...,PY,week
27,1494099268176527360,#UPDATE: #Russia has been building bridges and...,PY,week
28,1494099264636481540,"If Russia launches an invasion of Ukraine, mil...",PY,week
29,1494099264452059142,The Biden administration telling US citizens t...,PY,week
30,1494099263567061003,A senior Central African Republic military off...,PY,week
40,1494099262526873602,"NATO chief, US, and open source show no pullba...",PY,week
48,1494099259116904448,"Russia has added about 7,000 troops along Ukra...",PY,week
147,1494099258773065732,"As much as I loathe Putin, he has shown up Bid...",UC,week
148,1494099257388896258,"Maria Zakharova, the spokeswoman for Russia's ...",PN,week


In [202]:
month_merge['TimeFrame'] = 'month'
month_merge.head(10)

,id,tweet,rating,TimeFrame
1,1485039574556438530,@disclosetv Poppycock. Nothing to see here. Me...,DY,month
2,1485039574288150530,"In a bid to continue the attacks, @AbiyAhmedAl...",DY,month
3,1485039574027886593,BREAKING NEWS\n\nPUTIN PREPARING TO INSTALL PR...,DY,month
21,1485039571796692999,Hope these planes in Ukraine have better wheel...,DY,month
83,1485039570576023555,LONDON (AP) — British government accuses Russi...,DY,month
87,1485039570169303043,United States is creating this crisis by encou...,DY,month
88,1485039569825374209,Russia aims to replace Ukraine government with...,DY,month
93,1485039567912816640,What is Joe Biden’s plan to stop Russia from i...,DY,month
122,1485039566285422606,I think we should start paying far more attent...,DY,month
127,1485039565366706178,THERE SHOULD BE NO AMERICAN INVOLVEMENT IN UKR...,PY,month


In [223]:
total_merge = pd.concat([day_merge, week_merge, month_merge])
total_merge.head(10)
total_outpath = text_Dir.joinpath('ukraine_turked_total')


In [224]:
def preprocess_text(text):
 #Removes Numbers
 text = text.replace('\d+', '')
 str = text.lower()
 str = re.sub("'", "", str) # to avoid removing contractions in english
 str = re.sub("@[A-Za-z0-9_]+","", str)
 str = re.sub("#[A-Za-z0-9_]+","", str)
 str = re.sub(r'http\S+', '', str)
 str = re.sub('[()!?]', ' ', str)
 str = re.sub('\[.*?\]',' ', str)
 str = re.sub("[^a-z0-9]"," ", str)
 return str

In [225]:
total_merge['tweet'] = total_merge['tweet'].map(lambda x: p.clean(x))
total_merge['tweet'] = total_merge['tweet'].map(lambda x: preprocess_text(x))

print(len(total_merge))
total_merge.head(10)

2931


,id,tweet,rating,TimeFrame
0,1496273443754496003,no its the russia is communist right i lear...,UC,day
1,1496273551048986627,gt implying itll happen nrussia invading ukra...,DN,day
2,1496273438381686790,nothing but lies ukraine claims there are no ...,DN,day
3,1496273417372373000,just because anonymous us officials have been ...,DN,day
9,1496273394383441926,we are being played,DN,day
10,1496273370907828224,there is worry that will invade has denied i...,DN,day
11,1496273362737373186,i didnt realize the us put k troops on the bor...,DN,day
12,1496273357117206528,putin invades ukraine but not really via,DN,day
13,1496273331703599106,stop the lies about putin and russia this is ...,DN,day
14,1496273597719056386,this is when putins invasion of ukraine began ...,DY,day


In [226]:
total_merge.to_csv(total_outpath, header=False, index=False)

In [227]:
total_train = total_merge.sample(frac = .8)
#total_train.head(10)
total_test = total_merge.drop(total_train.index)
total_dev = total_test.sample(frac = .5)
total_test = total_test.drop(total_dev.index)
print(len(total_train))
print(len(total_test))
print(len(total_dev))
total_test.head(10)


2345
278
280


,id,tweet,rating,TimeFrame
279,1496273555163648002,the war between ukraine and russia started yea...,DY,day
327,1496273538868776964,a lot of americans who think that freedom figh...,DY,day
344,1496273533252558849,we havent said enough about the pact between p...,DY,day
367,1496273530073366529,after russia invaded in ukraine began centra...,DY,day
377,1496273529146331136,prof carlos he putin will keep everybody i...,DY,day
389,1496273521701441538,trump today endorsed putins invasion fuck him...,DY,day
560,1496273480907644930,announces sanctions on russia this is the b...,DY,day
624,1496273453879541762,former president donald trump on tuesday haile...,DY,day
631,1496273447563018244,this has very little if anything to do with na...,DY,day
663,1496273441351077889,the west didnt break vietnam after decades of ...,DY,day


# Bert Example

The downloaded textset is in tgz format. We can open it using `tarfile.open()` and then extract the csv files using the `extractall()` method:

Let's look at the the first 5 rows of the train and test textsets to understand the text we are dealing with:

We can see that in our textset a label of 1 means the review is bad while a label of 2 means the review is good.

Let's change this to a more standard pattern — 0 and 1 labels. Let's have a label 0 for the bad review and a label 1 for the good review:

In [228]:
total_train['rating'] = total_train['rating'] .replace(['DY', 'PY' , 'UC' , 'PN', 'DN'] , [2, 1, 0, -1, -2] )
total_dev['rating'] = total_dev['rating'] .replace(['DY', 'PY' , 'UC' , 'PN', 'DN'] , [2, 1, 0, -1, -2] )
total_test['rating'] = total_test['rating'] .replace(['DY', 'PY' , 'UC' , 'PN', 'DN'] , [2, 1, 0, -1, -2] )

In [229]:
total_train.head()

,id,tweet,rating,TimeFrame
31368,1494098041359458304,anyone who deviates from war propaganda is der...,-1,week
37823,1485038326151319561,russias purported plan to install pro kremlin ...,1,month
869,1496273326951514115,we promised to defend ukraine if they gave up ...,2,day
17032,1485039022904913922,i dont buy this russian nationalists dont wan...,2,month
82536,1485036896434667520,if russia installs a pro kremlin leadership i...,1,month


In [230]:
total_test.head()

,id,tweet,rating,TimeFrame
279,1496273555163648002,the war between ukraine and russia started yea...,2,day
327,1496273538868776964,a lot of americans who think that freedom figh...,2,day
344,1496273533252558849,we havent said enough about the pact between p...,2,day
367,1496273530073366529,after russia invaded in ukraine began centra...,2,day
377,1496273529146331136,prof carlos he putin will keep everybody i...,2,day


In [231]:
total_train['let'] = 'a'
total_train.head()

,id,tweet,rating,TimeFrame,let
31368,1494098041359458304,anyone who deviates from war propaganda is der...,-1,week,a
37823,1485038326151319561,russias purported plan to install pro kremlin ...,1,month,a
869,1496273326951514115,we promised to defend ukraine if they gave up ...,2,day,a
17032,1485039022904913922,i dont buy this russian nationalists dont wan...,2,month,a
82536,1485036896434667520,if russia installs a pro kremlin leadership i...,1,month,a


In [232]:
total_dev['let'] = 'a'
total_dev.head()

,id,tweet,rating,TimeFrame,let
6287,1494098969605062659,ukraines main military problem is that it has ...,0,week,a
7530,1494098909504970755,nato says satellite imagery shows that the rus...,1,week,a
2643,1496273553670524934,by design house and senate republicans are...,1,day,a
283,1496273552429006849,many ask where will putin stop nputin will s...,2,day,a
10103,1494098770493071361,why russia might invade ukraine fast facts yo...,0,week,a


In [233]:
total_train_final = total_train.loc[:, ['id','rating','let','tweet']]
total_train_final.head()

,id,rating,let,tweet
31368,1494098041359458304,-1,a,anyone who deviates from war propaganda is der...
37823,1485038326151319561,1,a,russias purported plan to install pro kremlin ...
869,1496273326951514115,2,a,we promised to defend ukraine if they gave up ...
17032,1485039022904913922,2,a,i dont buy this russian nationalists dont wan...
82536,1485036896434667520,1,a,if russia installs a pro kremlin leadership i...


In [234]:
total_dev_final = total_dev.loc[:, ['id','rating','let','tweet']]
total_test_final = total_test.loc[:, ['id','tweet']]

In [235]:
total_train_final.to_csv('ukraine_train.tsv', sep='\t', index=False, header=False)
total_dev_final.to_csv('ukraine_dev.tsv', sep='\t', index=False, header=False)
total_test_final.to_csv('ukraine_test.tsv', sep='\t', index=False, header=False)

### Making things BERT friendly
1. First let's make the text compliant with BERT:

    * Column 0: An ID for the row. (Required both for train and test text.)
    * Column 1: The class label for the row. (Required only for train text.)
    * Column 2: A column of the same letter for all rows — this is a throw-away column that we need to include because BERT expects it. (Required only for train text.)
    * Column 3: The text examples we want to classify. (Required both for train and test text.)

2. We need to split the files into the format expected by BERT: BERT comes with text loading classes that expects two files called train and dev for training. In addition, BERT’s text loading classes can also use a test file but it expects the test file to be unlabelled.


3. Once the text is in the correct format, we need to save the files as .tsv (BERT doesn't take .csv as input.)

In [10]:
# Creating training textframe according to BERT by adding the required columns
df_bert = pd.textFrame({
    'id':range(len(train_df)),
    'label':train_df[0],
    'alpha':['a']*train_df.shape[0],
    'text': train_df[1].replace(r'\n', ' ', regex=True)
})


# Splitting training text file into *train* and *dev*
df_bert_train, df_bert_dev = train_test_split(df_bert, test_size=0.01)

df_bert_train.head()

,id,label,alpha,text
229813,229813,0,a,I went in to Excellent Nails for a fill. The l...
159528,159528,1,a,IF YOU ARE A FRENCH FRY FREAK... YOU WILL LOVE...
76253,76253,0,a,There are few things that can go wrong when yo...
71226,71226,0,a,I stopped here out of annoyance of having to w...
268420,268420,1,a,"Red velvet cupcake here is superb, but it's a ..."


In [11]:
# Creating test textframe according to BERT
df_bert_test = pd.textFrame({
    'id':range(len(test_df)),
    'text': test_df[1].replace(r'\n', ' ', regex=True)
})

df_bert_test.head()

,id,text
0,0,"Contrary to other reviews, I have zero complai..."
1,1,Last summer I had an appointment to get new ti...
2,2,"Friendly staff, same starbucks fair you get an..."
3,3,The food is good. Unfortunately the service is...
4,4,Even when we didn't have a car Filene's Baseme...


In [12]:
# Saving textframes to .tsv format as required by BERT
df_bert_train.to_csv('text/train.tsv', sep='\t', index=False, header=False)
df_bert_dev.to_csv('text/dev.tsv', sep='\t', index=False, header=False)
df_bert_test.to_csv('text/test.tsv', sep='\t', index=False, header=False)

python run_classifier.py --task_name=cola --do_train=true --do_eval=true --do_predict=true --text_dir=./text/ --vocab_file=./all_bert_models/cased_L-24_H-1024_A-16/wwm_cased_L-24_H-1024_A-16/vocab.txt --bert_config_file=./all_bert_models/cased_L-24_H-1024_A-16/wwm_cased_L-24_H-1024_A-16/bert_config.json --init_checkpoint=./all_bert_models/cased_L-24_H-1024_A-16/wwm_cased_L-24_H-1024_A-16/bert_model.ckpt --max_seq_length=128 --train_batch_size=32 --learning_rate=2e-5 --num_train_epochs=3.0 --output_dir=./bert_output/ --do_lower_case=False